# Algorithm

In [58]:
using JuMP, Gurobi

# Gurobi
function kp_gurobi(values::Vector{Int}, weights::Vector{Int}, capacity::Int)
    num_items = length(weights)
    num_bins = length(weights)

    model = Model(optimizer_with_attributes(
        Gurobi.Optimizer,
        # "MIPGap" => 1e-4, # default
        "OutputFlag" =>0
    ))

    @variable(model, x[1:num_items], Bin)
    @objective(model, Max, sum(values[j] * x[j] for j in 1:num_items))
    @constraint(model, sum(weights[j] * x[j] for j in 1:num_items) <= capacity)

    optimize!(model)
    println("Objective value of Gurobi: ", objective_value(model))

    sol = Int64[]
    for (index, binary_val) in enumerate(value.(x))
        if binary_val == 1
            push!(sol, index)
        end
    end

    return objective_value(model), sol
end

kp_gurobi (generic function with 2 methods)

In [59]:
using Knapsacks

function kp_expandingcore(values::Vector{Int}, weights::Vector{Int}, capacity::Int)
    data = Knapsack(capacity, weights, values)

    opt, sol = solveKnapsack(data, :ExpandingCore)

    println("Objective value of Expandingcore: ", opt)
    return opt, sol
end

kp_expandingcore (generic function with 2 methods)

In [60]:
function kp_bruteforce(weights, values, capacity)
    n = length(weights)
    max_value = 0
    best_sol = Int64[]

    # Iterate over all possible combinations (2^n)
    for i in 0:(2^n - 1)
        total_weight = 0
        total_value = 0
        current_sol = Int64[]
        
        for j in 1:n
            if (i & (1 << (j - 1))) != 0 # bit masking
                total_weight += weights[j]
                total_value += values[j]
                push!(current_sol, j)
            end
        end
        
        if total_weight <= capacity && total_value > max_value
            max_value = total_value
            best_sol = current_sol
        end
    end

    println("Objective value of Bruteforce: ", max_value)
    
    return max_value, best_sol
end


kp_bruteforce (generic function with 1 method)

In [128]:
function kp_greedy(weights, values, capacity)
    n = length(weights)
    items = [(values[i] / weights[i], i) for i in 1:n]
    
    sort!(items, by = x -> -x[1])
    #println(items)
    
    total_weight = 0
    total_value = 0
    solution = Int64[]
    
    for (_, i) in items
        if total_weight + weights[i] <= capacity
            total_weight += weights[i]
            total_value += values[i]
            push!(solution, i)
        end
    end
    
    println("Objective value of Greedy: ", total_value)
    
    return total_value, solution
end

kp_greedy (generic function with 1 method)

# Problem

### n = 6 

In [174]:
import Random
Random.seed!(2)

n = 6
values = rand(1:10, n)
weights = rand(1:20, n)
capacity = 50

println("Values: ", values)
println("Weights: ", weights)

Values: [1, 7, 8, 9, 3, 9]
Weights: [14, 9, 16, 15, 20, 15]


In [175]:
println(" 1. Gurobi")
@time opt1, sol1 = kp_gurobi(values, weights, capacity)
println("Index of Selected items: ", sol1)

println("\n 2. ExpandingCore")
@time opt2, sol2 = kp_expandingcore(values, weights, capacity)
println("Index of Selected items: ", sol2)

println("\n 3. Bruteforce")
@time opt3, sol3 = kp_bruteforce(weights, values, capacity)
println("Index of Selected items: ", sol3)

println("\n 4. Greedy")
@time opt4, sol4 = kp_greedy(weights, values, capacity)
println("Index of Selected items: ", sort!(sol4))

 1. Gurobi
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Objective value of Gurobi: 26.0
  0.003197 seconds (2.74 k allocations: 79.578 KiB)
Index of Selected items: [3, 4, 6]

 2. ExpandingCore
Objective value of Expandingcore: 26
  0.000048 seconds (81 allocations: 4.477 KiB)
Index of Selected items: [3, 4, 6]

 3. Bruteforce
Objective value of Bruteforce: 26
  0.000040 seconds (169 allocations: 9.961 KiB)
Index of Selected items: [3, 4, 6]

 4. Greedy
Objective value of Greedy: 25
  0.000124 seconds (167 allocations: 13.648 KiB)
Index of Selected items: [2, 4, 6]


### n = 10

In [159]:
import Random
Random.seed!(5)

n = 10
values = rand(1:100, n)
weights = rand(1:100, n)
capacity = 100

100

In [160]:
println(" 1. Gurobi")
@time opt1, sol1 = kp_gurobi(values, weights, capacity)
println("Index of Selected items: ", sol1)

println("\n 2. ExpandingCore")
@time opt2, sol2 = kp_expandingcore(values, weights, capacity)
println("Index of Selected items: ", sol2)

println("\n 3. Bruteforce")
@time opt3, sol3 = kp_bruteforce(weights, values, capacity)
println("Index of Selected items: ", sol3)

println("\n 4. Greedy")
@time opt4, sol4 = kp_greedy(weights, values, capacity)
println("Index of Selected items: ", sort!(sol4))

 1. Gurobi
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Objective value of Gurobi: 277.0
  0.002352 seconds (2.90 k allocations: 89.281 KiB)
Index of Selected items: [2, 6, 8, 10]

 2. ExpandingCore
Objective value of Expandingcore: 277
  0.000057 seconds (143 allocations: 10.273 KiB)
Index of Selected items: [2, 6, 8, 10]

 3. Bruteforce
Objective value of Bruteforce: 277
  0.000132 seconds (2.10 k allocations: 148.602 KiB)
Index of Selected items: [2, 6, 8, 10]

 4. Greedy
Objective value of Greedy: 267
  0.000048 seconds (44 allocations: 1.352 KiB)
Index of Selected items: [3, 6, 7, 8, 10]


In [161]:
import Random
Random.seed!(1234)

n = 10
values = rand(1:100, n)
weights = rand(1:100, n)
capacity = 100

100

In [162]:
println(" 1. Gurobi")
@time opt1, sol1 = kp_gurobi(values, weights, capacity)
println("Index of Selected items: ", sol1)

println("\n 2. ExpandingCore")
@time opt2, sol2 = kp_expandingcore(values, weights, capacity)
println("Index of Selected items: ", sol2)

println("\n 3. Bruteforce")
@time opt3, sol3 = kp_bruteforce(weights, values, capacity)
println("Index of Selected items: ", sol3)

println("\n 4. Greedy")
@time opt4, sol4 = kp_greedy(weights, values, capacity)
println("Index of Selected items: ", sort!(sol4))

 1. Gurobi
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Objective value of Gurobi: 192.0
  0.002190 seconds (2.96 k allocations: 91.031 KiB)
Index of Selected items: [3, 4, 8]

 2. ExpandingCore
Objective value of Expandingcore: 192
  0.000058 seconds (108 allocations: 7.180 KiB)
Index of Selected items: [3, 4, 8]

 3. Bruteforce
Objective value of Bruteforce: 192
  0.000136 seconds (2.10 k allocations: 148.570 KiB)
Index of Selected items: [3, 4, 8]

 4. Greedy
Objective value of Greedy: 192
  0.000071 seconds (44 allocations: 1.320 KiB)
Index of Selected items: [3, 4, 8]


### n = 20

In [163]:
import Random
Random.seed!(1234)

n = 20
values = rand(1:100, n)
weights = rand(1:100, n)
capacity = 200

200

In [134]:
println(" 1. Gurobi")
@time opt1, sol1 = kp_gurobi(values, weights, capacity)
println("Index of Selected items: ", sol1)

println("\n 2. ExpandingCore")
@time opt2, sol2 = kp_expandingcore(values, weights, capacity)
println("Index of Selected items: ", sol2)

println("\n 3. Bruteforce")
@time opt3, sol3 = kp_bruteforce(weights, values, capacity)
println("Index of Selected items: ", sol3)

println("\n 4. Greedy")
@time opt4, sol4 = kp_greedy(weights, values, capacity)
println("Index of Selected items: ", sort!(sol4))

 1. Gurobi
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Objective value of Gurobi: 568.0
  0.003446 seconds (3.04 k allocations: 99.656 KiB)
Index of Selected items: [1, 4, 6, 7, 12, 17, 18, 20]

 2. ExpandingCore
Objective value of Expandingcore: 568
  0.000132 seconds (296 allocations: 24.352 KiB)
Index of Selected items: [1, 4, 6, 7, 12, 17, 18, 20]

 3. Bruteforce
Objective value of Bruteforce: 568
  0.226046 seconds (2.88 M allocations: 395.434 MiB, 21.81% gc time)
Index of Selected items: [1, 4, 6, 7, 12, 17, 18, 20]

 4. Greedy
Objective value of Greedy: 568
  0.000047 seconds (49 allocations: 1.977 KiB)
Index of Selected items: [1, 4, 6, 7, 12, 17, 18, 20]


In [135]:
import Random
Random.seed!(0)

n = 20
values = rand(1:100, n)
weights = rand(1:100, n)
capacity = 200

200

In [136]:
println(" 1. Gurobi")
@time opt1, sol1 = kp_gurobi(values, weights, capacity)
println("Index of Selected items: ", sol1)

println("\n 2. ExpandingCore")
@time opt2, sol2 = kp_expandingcore(values, weights, capacity)
println("Index of Selected items: ", sol2)

println("\n 3. Bruteforce")
@time opt3, sol3 = kp_bruteforce(weights, values, capacity)
println("Index of Selected items: ", sol3)

println("\n 4. Greedy")
@time opt4, sol4 = kp_greedy(weights, values, capacity)
println("Index of Selected items: ", sort!(sol4))

 1. Gurobi
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Objective value of Gurobi: 479.0
  0.003549 seconds (3.04 k allocations: 99.953 KiB)
Index of Selected items: [1, 3, 5, 12, 13, 14, 15, 16]

 2. ExpandingCore
Objective value of Expandingcore: 479
  0.000048 seconds (143 allocations: 9.992 KiB)
Index of Selected items: [1, 3, 5, 12, 13, 14, 15, 16]

 3. Bruteforce
Objective value of Bruteforce: 479
  0.213490 seconds (2.88 M allocations: 395.434 MiB, 17.49% gc time)
Index of Selected items: [1, 3, 5, 12, 13, 14, 15, 16]

 4. Greedy
Objective value of Greedy: 454
  0.000054 seconds (45 allocations: 1.914 KiB)
Index of Selected items: [1, 3, 4, 5, 12, 13, 15, 16]


### n = 25

In [137]:
import Random
Random.seed!(1234)

n = 25
values = rand(1:100, n)
weights = rand(1:100, n)
capacity = 250

250

In [138]:
println(" 1. Gurobi")
@time opt1, sol1 = kp_gurobi(values, weights, capacity)
println("Index of Selected items: ", sol1)

println("\n 2. ExpandingCore")
@time opt2, sol2 = kp_expandingcore(values, weights, capacity)
println("Index of Selected items: ", sol2)

println("\n 3. Bruteforce")
@time opt3, sol3 = kp_bruteforce(weights, values, capacity)
println("Index of Selected items: ", sol3)

println("\n 4. Greedy")
@time opt4, sol4 = kp_greedy(weights, values, capacity)
println("Index of Selected items: ", sort!(sol4))

 1. Gurobi
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Objective value of Gurobi: 630.0
  0.003596 seconds (3.16 k allocations: 104.922 KiB)
Index of Selected items: [1, 2, 7, 8, 13, 15, 17, 19, 20, 22]

 2. ExpandingCore
Objective value of Expandingcore: 630
  0.000057 seconds (182 allocations: 13.508 KiB)
Index of Selected items: [1, 2, 7, 8, 13, 15, 17, 19, 20, 22]

 3. Bruteforce
Objective value of Bruteforce: 630
  7.994399 seconds (98.86 M allocations: 14.434 GiB, 17.58% gc time)
Index of Selected items: [1, 2, 7, 8, 13, 15, 17, 19, 20, 22]

 4. Greedy
Objective value of Greedy: 630
  0.000061 seconds (50 allocations: 2.492 KiB)
Index of Selected items: [1, 2, 7, 8, 13, 15, 17, 19, 20, 22]


### n = 30

In [139]:
import Random
Random.seed!(1234)

n = 30
values = rand(1:100, n)
weights = rand(1:100, n)
capacity = 300

300

In [140]:
println(" 1. Gurobi")
@time opt1, sol1 = kp_gurobi(values, weights, capacity)
println("Index of Selected items: ", sol1)

println("\n 2. ExpandingCore")
@time opt2, sol2 = kp_expandingcore(values, weights, capacity)
println("Index of Selected items: ", sol2)

println("\n 3. Bruteforce")
@time opt3, sol3 = kp_bruteforce(weights, values, capacity)
println("Index of Selected items: ", sol3)

println("\n 4. Greedy")
@time opt4, sol4 = kp_greedy(weights, values, capacity)
println("Index of Selected items: ", sort!(sol4))

 1. Gurobi
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Objective value of Gurobi: 704.0
  0.004363 seconds (3.28 k allocations: 109.234 KiB)
Index of Selected items: [2, 3, 7, 8, 10, 12, 16, 17, 28, 30]

 2. ExpandingCore
Objective value of Expandingcore: 704
  0.000051 seconds (257 allocations: 20.383 KiB)
Index of Selected items: [2, 3, 7, 8, 10, 12, 16, 17, 28, 30]

 3. Bruteforce
Objective value of Bruteforce: 704
277.427623 seconds (3.21 G allocations: 477.291 GiB, 16.87% gc time)
Index of Selected items: [2, 3, 7, 8, 10, 12, 16, 17, 28, 30]

 4. Greedy
Objective value of Greedy: 704
  0.000056 seconds (50 allocations: 2.586 KiB)
Index of Selected items: [2, 3, 7, 8, 10, 12, 16, 17, 28, 30]
